In [1]:
# merge and rename
import shutil
import os
path = "/msl/home/kelly08385/Pictures/Fast_defect/8800_0307/data"
despath = "/msl/home/kelly08385/Pictures/Fast_defect/8800_0307/JPEGImages"
for root,dirs,files in os.walk(path):
    for file in files:
        #dir = root.split('/')[-1]
        shutil.copy(os.path.join(root,file),os.path.join(os.path.join(despath,file)))
        #os.rename(os.path.join(root,file),os.path.join(despath, dir, file[:-4]+'.jpg'))

In [4]:
def mkdir(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [16]:
## use original image and crop image to find xml 

import xml.etree.cElementTree as ET
import cv2
import os
import numpy as np
import shutil

crop_path = "/msl/home/kelly08385/Pictures/Fast_defect/1251/crop"
des_crop_path = "/msl/home/kelly08385/Pictures/Fast_defect/1251/train_crop"
ori_path = "/msl/home/kelly08385/Pictures/Fast_defect/1251/ori"
des_path = "/msl/home/kelly08385/Pictures/Fast_defect/1251/data"
des_path2 = "/msl/home/kelly08385/Pictures/Fast_defect/1251/JPEGImages"
xml_path = "/msl/home/kelly08385/Pictures/Fast_defect/1251/Annotations"

#quiz_path = "/home/kelly/Pictures/defect/0312_work/9300/quiz"
        

In [ ]:
# use xml to catch picture
ori_path = "/home/kelly/Pictures/defect/0322_work/4800/no_rec"
crop_path = "/home/kelly/Pictures/defect/0322_work/4800/rec"
xml_path = "/home/kelly/Pictures/fastrcnn/4800_0323/result/Annotations"
des_path = "/home/kelly/Pictures/fastrcnn/4800_0323/data"

for root,dirs,files in os.walk(crop_path):
    for file in files:
        dir = root.split('/')[-1]
        print(os.path.join(ori_path,dir,file[:-4]+".jpg"))
        print(os.path.join(xml_path,file[:-4]+".xml"))
        desPath = os.path.join(des_path,dir)
        print(desPath)
#         if os.path.exists(os.path.join(ori_path,dir,file[:-4]+".jpg")) and\
#                           os.path.exists(os.path.join(xml_path,file[:-4]+".xml")):
#             mkdir(desPath)
#             shutil.move(os.path.join(ori_path,dir,file[:-4]+".jpg"),os.path.join(des_path,dir,file[:-4]+".jpg"))

In [17]:
for root,dirs,files in os.walk(crop_path):
    dir = root.split("/")[-1]
    for file in files:
        mkdir(os.path.join(des_path,dir))
        mkdir(os.path.join(des_crop_path,dir))
        if os.path.exists(os.path.join(ori_path,dir,file[:-4]+".jpg")):

            try:
                cropImg = cv2.imread(os.path.join(root,file))
                cropImg = cv2.cvtColor(cropImg, cv2.COLOR_BGR2GRAY)
                oriImg = cv2.imread(os.path.join(ori_path,dir,file[:-4]+".jpg"))
                oriImg = cv2.cvtColor(oriImg, cv2.COLOR_BGR2GRAY)
                
                result = cv2.matchTemplate(oriImg, cropImg, cv2.TM_CCOEFF_NORMED) 
                x_and_y = np.unravel_index(result.argmax(),result.shape)
                print(x_and_y)

                tree = ET.parse('/msl/home/kelly08385/Pictures/trained_defect/total/Annotations/2300AAHL5AHYC14C100007-03.xml')
                tree_root = tree.getroot()
                print(tree_root)
                change_dict = {"filename":file,"width":oriImg.shape[1],"height":oriImg.shape[0],"xmin":x_and_y[1],"ymin":x_and_y[0],\
                               "xmax":x_and_y[1]+cropImg.shape[1],"ymax":x_and_y[0]+cropImg.shape[0],"name":root.split("/")[-1]}
                for key in change_dict.keys():
                    for e in tree_root.iter(key):
                        e.text = str(change_dict[key])
                shutil.copy(os.path.join(ori_path,dir,file),os.path.join(des_path2,file))
                shutil.move(os.path.join(ori_path,dir,file),os.path.join(des_path,dir,file))
                shutil.move(os.path.join(root,file),os.path.join(des_crop_path,dir,file))
                tree.write(os.path.join(xml_path,file[:-4]+".xml"))
            except:
                print("contunue")

(59, 33)
<Element 'annotation' at 0x7f67067ddbd8>
(277, 693)
<Element 'annotation' at 0x7f67067dd958>
(239, 565)
<Element 'annotation' at 0x7f67c6c1a728>
(387, 623)
<Element 'annotation' at 0x7f67c6c1ad18>
(348, 551)
<Element 'annotation' at 0x7f67067ddc28>
(73, 498)
<Element 'annotation' at 0x7f67c6d03f48>
(481, 627)
<Element 'annotation' at 0x7f67c6d034f8>
(273, 609)
<Element 'annotation' at 0x7f6706860f48>
(335, 599)
<Element 'annotation' at 0x7f6707467778>
(279, 501)
<Element 'annotation' at 0x7f67c6d03598>
(371, 579)
<Element 'annotation' at 0x7f6706860f48>
(310, 585)
<Element 'annotation' at 0x7f67c6c1a548>
(173, 481)
<Element 'annotation' at 0x7f67c6c1a728>
(251, 767)
<Element 'annotation' at 0x7f6706860f48>
(272, 554)
<Element 'annotation' at 0x7f6707467778>
(579, 531)
<Element 'annotation' at 0x7f67c6c1ad18>
(194, 637)
<Element 'annotation' at 0x7f6706860f48>
(245, 515)
<Element 'annotation' at 0x7f67c6d03f48>
(123, 483)
<Element 'annotation' at 0x7f67c6d03138>
(44, 6)
<Elemen

(261, 585)
<Element 'annotation' at 0x7f67068bdf48>
(323, 637)
<Element 'annotation' at 0x7f67c6d28f98>
(309, 673)
<Element 'annotation' at 0x7f67c6d03598>
(139, 547)
<Element 'annotation' at 0x7f67c6d03b88>
(261, 645)
<Element 'annotation' at 0x7f67c6d28f98>
(359, 637)
<Element 'annotation' at 0x7f67068bdc78>
(324, 557)
<Element 'annotation' at 0x7f67c6d03138>
(385, 671)
<Element 'annotation' at 0x7f67c6d03d18>
(65, 619)
<Element 'annotation' at 0x7f67068bd958>
(9, 289)
<Element 'annotation' at 0x7f67068bde08>
(245, 493)
<Element 'annotation' at 0x7f67c6d03e08>
(419, 729)
<Element 'annotation' at 0x7f67c6d03bd8>
(218, 1028)
<Element 'annotation' at 0x7f67068bdb38>
(235, 495)
<Element 'annotation' at 0x7f67068bdc28>
(153, 571)
<Element 'annotation' at 0x7f67c6d033b8>
(441, 722)
<Element 'annotation' at 0x7f67c6d03f98>
(311, 713)
<Element 'annotation' at 0x7f67068bdea8>
(293, 585)
<Element 'annotation' at 0x7f67068bdcc8>
(289, 129)
<Element 'annotation' at 0x7f67c6d03e58>
(389, 689)
<El

(303, 540)
<Element 'annotation' at 0x7f6706860f48>
(192, 592)
<Element 'annotation' at 0x7f67c6d03138>
(556, 801)
<Element 'annotation' at 0x7f67c6d033b8>
(482, 730)
<Element 'annotation' at 0x7f6706860f48>
(450, 690)
<Element 'annotation' at 0x7f67068bd9f8>
(231, 495)
<Element 'annotation' at 0x7f6706872e58>
(193, 677)
<Element 'annotation' at 0x7f67c6d033b8>
(209, 576)
<Element 'annotation' at 0x7f67068bdc78>
(382, 615)
<Element 'annotation' at 0x7f6706872f98>
(579, 716)
<Element 'annotation' at 0x7f67c6d03638>
(367, 620)
<Element 'annotation' at 0x7f67c6d03d18>
(380, 702)
<Element 'annotation' at 0x7f6706872f98>
(300, 617)
<Element 'annotation' at 0x7f67068bdef8>
(319, 765)
<Element 'annotation' at 0x7f67c6d03e58>
(304, 538)
<Element 'annotation' at 0x7f6706872f98>
(471, 734)
<Element 'annotation' at 0x7f6706860f48>
(356, 638)
<Element 'annotation' at 0x7f67068bdea8>
(238, 528)
<Element 'annotation' at 0x7f6706872f98>
(364, 746)
<Element 'annotation' at 0x7f6706872e58>
(453, 724)
<

# Use A little train size to do faster RCNN

In [ ]:
# use picture to catch xml
import shutil 
pic_path = "/home/kelly/Pictures/fastrcnn/8800_0307/data"
xml_path = "/home/kelly/Pictures/defect/0307_work/xml"
des_xml_path = "/home/kelly/Pictures/fastrcnn/8800_0307/result/Annotations"

for root, dirs,files in os.walk(pic_path):
    for file in files:
        shutil.copy(os.path.join(xml_path,file[:-4]+".xml"),os.path.join(des_xml_path,file[:-4]+".xml"))

In [ ]:
# select the untrain image 
import shutil
original_pic_path = "/home/kelly/Pictures/fastrcnn/8800_0124/picture"
trained_pic_path = "/home/kelly/Pictures/fastrcnn/8800_0302_100/result2/data"
des_path = "/home/kelly/Pictures/fastrcnn/8800_0302_100/result2/demo"

for root, dirs, files in os.walk(original_pic_path):
    for file in files:
        dir = root.split("/")[-1]
        dir_path = os.path.join(trained_pic_path,dir)
        if not os.path.exists(os.path.join(des_path,dir)):
            os.mkdir(os.path.join(des_path,dir))
        trained_file = os.listdir(dir_path)
        if not os.path.exists(os.path.join(des_path,dir)):
            os.mkdir(os.path.join(des_path,dir))
        if not file in trained_file:
            shutil.copy(os.path.join(root,file),os.path.join(des_path,dir,file))


---------------------------------------------------------------------------------------

In [7]:
# find red rec
from scipy.misc import imread, imsave
import numpy as np
from skimage.segmentation import clear_border
from skimage.measure import regionprops, label

def size(a, b=0, nargout=1):
    s = np.asarray(a).shape
    if s is ():
        return 1 if b else (1,)*nargout
    # a is not a scalar
    try:
        if b:
            return s[b-1]
        elif nargout > 1:
            return s
    except Exception:
        return 1
    return  1
 
def im2double(im):
    info = np.iinfo(im.dtype)
    return im.astype(np.float) / info.max

def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.299, 0.587, 0.114])

def double(img):
    return img.astype(np.float)

def getImgInRedRect(imgPath):
        ret = ''
        path_arr = imgPath.split(os.path.sep)
        rawImg = imread(imgPath)
        if size(rawImg,3) < 3:
            return ret
#         output = os.path.abspath(os.path.join(targetPath,'redRec'))
#         for index, path_ in enumerate(path_arr):
#             if path_ not in sourcePath.split(os.path.sep):
#                 output = os.path.join(output, path_)
 
        tmpImg = np.copy(rawImg)
        tmpImg = im2double(tmpImg)
        tmpImg[:,:,0] = tmpImg[:,:,0] > 0.8
        tmpImg[:,:,1] = tmpImg[:,:,1] < 0.2
        tmpImg[:,:,2] = tmpImg[:,:,2] < 0.2
        tmpImg = rgb2gray(tmpImg)
        tmpImg = double(tmpImg > 0.8)
        bw = tmpImg > 0.8
        bw = bw.astype(int)
        cleared = clear_border(bw)
        stats = regionprops(label(cleared))
        flag = False
        #print (imgPath)
        #print(output, output2, output3)
        for currentStats in stats:
            minr, minc, maxr, maxc = currentStats.bbox
            if currentStats.area > 1000 and (maxr-minr) > 25 and (maxc-minc) > 25: # red rectangle
                x = int(max(round(minr + 10), 1))
                y = int(min(round(maxc - 10), size(rawImg,2)))
                crop = rawImg[x:int(min(round(maxr - 10), size(rawImg,1))), int(max(round(minc + 10) , 1)):y,:]
                flag = True
                returnX = x
                returnY = int(max(round(minc + 10) , 1))
                #imsave(output, crop)
        if flag:
            return crop
        else:
            return None
                #print ("red rec:", output)
 


In [11]:
# run the faster rcnn demo
import cv2,os,shutil
import xml.etree.cElementTree as ET


# use demooutput to find xml
# use rec to find no rec

def create_xml(ori_path,root,file,xml_path,data_path,data_path2,des_rec):
    
    imgPath = os.path.join(root,file)
    ori_path = os.path.join(ori_path,root.split('/')[-1])
    # 原圖不存在
    if not os.path.exists(os.path.join(ori_path,file[:-4]+".jpg")):
        return
    oriImg = cv2.imread(os.path.join(ori_path,file[:-4]+".jpg"))
    oriImg = cv2.cvtColor(oriImg, cv2.COLOR_BGR2GRAY)
        
    #xml_path = os.path.join(xml_path,root.split('/')[-1])
    data_path = os.path.join(data_path,root.split('/')[-1])
    des_rec_path = os.path.join(des_rec,root.split('/')[-1])
    mkdir(des_rec_path)
    mkdir(data_path)

    cropImg = getImgInRedRect(imgPath)
    
    if not cropImg is None :
        cropImg = cv2.cvtColor(cropImg, cv2.COLOR_BGR2GRAY)
        oriImg.astype(np.uint8)
        #print(cropImg)
        #cropImg = cropImg[:,:,:-1]
        #print(cropImg)
        result = cv2.matchTemplate(oriImg, cropImg, cv2.TM_CCOEFF_NORMED) 
        x_and_y = np.unravel_index(result.argmax(),result.shape)
        print(x_and_y)

        tree = ET.parse('/msl/home/kelly08385/Pictures/trained_defect/total/Annotations/2300AAHL5AHYC14C100007-03.xml')
        tree_root = tree.getroot()
        change_dict = {"filename":"2300AAHL5AHYC14C100007-03.xml","width":oriImg.shape[1],"height":oriImg.shape[0],"xmin":x_and_y[1],"ymin":x_and_y[0],\
                                       "xmax":x_and_y[1]+cropImg.shape[1],"ymax":x_and_y[0]+cropImg.shape[0],"name":root.split('/')[-1]}

        for key in change_dict.keys():
            for e in tree_root.iter(key):
                e.text = str(change_dict[key])
                tree.write(os.path.join(xml_path,file[:-4]+".xml"))
        shutil.copy(os.path.join(ori_path,file),os.path.join(data_path2,file))
        shutil.move(os.path.join(ori_path,file),os.path.join(data_path,file))
        shutil.move(imgPath,os.path.join(des_rec_path,file))


In [15]:
# use demooutput to create xml
# use rec to find no rec



ori_path = "/msl/home/kelly08385/Pictures/Fast_defect/8800/ori"
rec_path = "/msl/home/kelly08385/Pictures/Fast_defect/8800/rec"
xml_path = "/msl/home/kelly08385/Pictures/Fast_defect/8800/Annotations"
data_path = "/msl/home/kelly08385/Pictures/Fast_defect/8800/data"
data_path2 = "/msl/home/kelly08385/Pictures/Fast_defect/8800/JPEGImages"
des_rec = "/msl/home/kelly08385/Pictures/Fast_defect/8800/train_rec"

for root, dirs, files in os.walk(rec_path):
    for file in files:
        print(file)
        create_xml(ori_path,root,file,xml_path,data_path,data_path2,des_rec)


8800TL83H0MAK15C100001-01.jpg


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:34: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.


(310, 563)
8800TL8334CAJ1A1100001-01.jpg
(214, 448)
8800TL83H3EBB18B100001-01.jpg
8800TL83K2GAK149100001-01.jpg
(128, 251)
8800TL83J0WBB11E100002-01.jpg
(160, 584)
8800TL83J1CBB151100001-01.jpg
(554, 676)
8800TL83R31AH1BB100001-01.jpg
(463, 637)
8800TL83M3RAF135100001-01.jpg
(362, 564)
8800TL83E0EAE127100001-01.jpg
(380, 686)
8800TL83N1AAL105100001-01.jpg
8800TL83N0SAQ199100001-01.jpg
(186, 223)
8800TL83G05AM1A3100001-01.jpg
(239, 572)
8800TL83D3BAA14D100001-01.jpg
8800TL83H1KAV1B6100003-01.jpg
(238, 525)
8800TL83H34AJ10B100001-01.jpg
(72, 507)
8800TL83J02AY122100001-01.jpg
8800TL83R33AD18M100001-01.jpg
(205, 24)
8800TL83948AD1G3100001-01.jpg
(694, 642)
8800TL83M3PAC179100001-01.jpg
(326, 606)
8800TL83K2RAG13L100001-01.jpg
(94, 540)
8800TL83H3CAB1A5100001-01.jpg
(483, 603)
8800TL83H0TAA11B100002-01.jpg
8800TL83E2RBB18B100001-01.jpg
(453, 686)
8800TL83M0CAD1A9100001-01.jpg
(378, 606)
8800TL83H0RAU164100001-01.jpg
(316, 532)
8800TL83N18AR1A1100001-01.jpg
(296, 494)
8800TL83E2HAV1DJ100001

(19, 736)
8800TL83N0WAY151100001-01.jpg
(314, 479)
8800TL83M3LAL126100001-01.jpg
(357, 499)
8800TL83D3BAJ1CB100001-01.jpg
8800TL83M3RAT116100001-01.jpg
(190, 498)
8800TL83H3KAM1CF100001-01.jpg
8800TL83H3CAY1BL100001-01.jpg
8800TL83H0TAB16J100001-01.jpg
(359, 435)
8800TL83M36AA195100001-01.jpg
(165, 543)
8800TL83E1PAW1BC100001-01.jpg
(488, 491)
8800TL83N18AH139100001-01.jpg
(307, 517)
8800TL8340TAW1C4100001-01.jpg
8800TL83L1DBD1AG100001-01.jpg
(327, 620)
8800TL83H0TAA11B100001-01.jpg
(378, 651)
8800TL83H1JAD10J100001-01.jpg
(161, 417)
8800TL83M36AD1BF100001-01.jpg
(412, 428)
8800TL83J13AA1AJ100001-01.jpg
8800TL83M0CAK18A100001-01.jpg
(298, 578)
8800TL83R31AR19J100001-01.jpg
(354, 314)
8800TL83K2GAT1D5100001-01.jpg
(368, 583)
8800TL83E1PAK147100001-01.jpg
(412, 560)
8800TL83H31BD15L100001-01.jpg
(429, 612)
8800TL83G1MAP19J100001-01.jpg
8800TL83E2JAA11E100001-01.jpg
(318, 569)
8800TL83E49AZ1B1100001-01.jpg
(208, 559)
8800TL83E0EAZ15F100001-01.jpg
(288, 607)
8800TL83J2DAR17A100001-01.jpg
8

(426, 619)
8800TL8340TAB1E8100001-01.jpg
(394, 545)
8800TL83D1QAV14J100001-01.jpg
(311, 568)
8800TL83E1LBC168100001-01.jpg
(290, 532)
8800TL83M3KAH126100001-01.jpg
(327, 721)
8800TL83Q16AS11E100001-01.jpg
(345, 538)
8800TL83H1JBB15K100001-01.jpg
(464, 428)
8800TL83F0DAK108100001-01.jpg
(190, 703)
8800TL83H0VAK11A100001-01.jpg
(337, 445)
8800TL83H1GAG19B100001-01.jpg
(320, 630)
8800TL83M3KAP141100001-01.jpg
(402, 664)
8800TL83D0CAW18B100001-01.jpg
(235, 600)
8800TL83K2QBD123100001-01.jpg
8800TL83J17AB1AB100003-01.jpg
(285, 595)
8800TL83J13AQ157100001-01.jpg
(188, 624)
8800TL83J2CAY165100001-01.jpg
(282, 620)
8800TL83J1DAU16A100001-01.jpg
(274, 541)
8800TL83G2HAW139100001-01.jpg
8800TL83N2JAU17G100001-01.jpg
(649, 529)
8800TL83J2CAD18C100001-01.jpg
(320, 488)
8800TL83C5MAX1C4100001-01.jpg
8800TL83J1DAT13E100001-01.jpg
(455, 588)
8800TL83H0VBD1B4100001-01.jpg
(88, 114)
8800TL83N18AT17F100001-01.jpg
(64, 588)
8800TL83G1KAW141100001-01.jpg
(508, 613)
8800TL83N2FAD10D100001-01.jpg
(82, 512)


...

(259, 708)
8800TL83H31AD16J100001-01.jpg
(314, 389)
8800TL83K2MBA156100001-01.jpg
(381, 696)
8800TL83F2RAL1BL100001-01.jpg
(415, 450)
8800TL83M0XBC1A2100001-01.jpg
(435, 584)
8800TL83N0SAK19A100001-01.jpg
(348, 542)
8800TL83M0UAD1D1100004-01.jpg
(503, 428)
8800TL83H3AAM1BC100001-01.jpg
(458, 599)
8800TL83K2RAF12E100001-01.jpg
(226, 653)
8800TL83J2CAR13F100001-01.jpg
(291, 634)
8800TL83M0XAB16J100001-01.jpg
(349, 623)
8800TL83H38AX118100001-01.jpg
(379, 573)
8800TL83J10AP13N100001-01.jpg
(543, 506)
8800TL83Q16AN122100001-01.jpg
8800TL83K2LAS17F100001-01.jpg
(342, 643)
8800TL83M3LAN121100001-01.jpg
(392, 550)
8800TL83N2JBD16G100001-01.jpg
(474, 682)
8800TL83J12AZ1B1100002-01.jpg
(377, 682)
8800TL83K0YAK191100001-01.jpg
(622, 614)
8800TL83J1CAW161100001-01.jpg
(513, 689)
8800TL83F2RAT19C100001-01.jpg
(391, 640)
8800TL83G1LBA1B8100001-01.jpg
(476, 555)
8800TL83H1QAE167100001-01.jpg
(330, 620)
8800TL83R32BA14H100001-01.jpg
(530, 483)
8800TL83H0MAV121100001-01.jpg
(304, 491)
8800TL83K2MAD18J

(324, 577)
8800TL83R31AX1AC100001-01.jpg
(536, 608)
8800TL83J19AX1BJ100001-01.jpg
(464, 604)
8800TL83N0SAC189100001-01.jpg
(334, 205)
8800TL83M3KAS1B6100001-01.jpg
(549, 682)
8800TL83M0UAG115100001-01.jpg
(374, 801)
8800TL83S0CAN10L100001-01.jpg
8800TL83E2HAF152100001-01.jpg
(222, 597)
8800TL83H1XBC14F100001-03.jpg
(438, 558)
8800TL83M3MBB1B8100001-01.jpg
(592, 626)
8800TL83M0UAH124100001-01.jpg
(228, 810)
8800TL83R31AT11B100001-01.jpg
(51, 654)
8800TL83J29AP122100001-01.jpg
(316, 600)
8800TL83M3MAT14G100001-01.jpg
(293, 584)
8800TL83Q16AP154100001-01.jpg
8800TL83H1EAC17E100001-01.jpg
(558, 513)
8800TL83K2LAH1C1100001-01.jpg
(500, 682)
8800TL83M0UAT17B100001-01.jpg
(314, 721)
8800TL83H0PAH1AH100001-01.jpg
(459, 623)
8800TL83J1CBC12B100001-01.jpg
(211, 680)
8800TL83N0WAU16D100001-01.jpg
(282, 527)
8800TL83H3CAA18A100001-01.jpg
8800TL83G2NAM101100001-01.jpg
(458, 660)
8800TL83F2TAE148100001-01.jpg
(278, 436)
8800TL83M3KAA10L100001-01.jpg
(176, 211)
8800TL83N0VBB107100001-01.jpg
(307, 615

(510, 541)
8800TL83K2HAF15J100001-01.jpg
(328, 264)
8800TL83N0SAZ1A1100002-01.jpg
(785, 778)
8800TL83M0XAP118100001-01.jpg
(221, 434)
8800TL83M3KAF121100001-01.jpg
(447, 675)
8800TL83K2MAA119100001-01.jpg
(135, 19)
8800TL83C04AN127100001-01.jpg
(285, 408)
8800TL83F2RAB13L100001-01.jpg
(54, 315)
8800TL83U1UAS15F100001-01.jpg
(218, 652)
8800TL83J29AY113100001-01.jpg
(505, 648)
8800TL83K2QAG17B100001-01.jpg
(590, 662)
8800TL83N0WAS11J100001-01.jpg
(155, 466)
8800TL83H1QAH12A100001-01.jpg
(387, 718)
8800TL83H34AW134100001-01.jpg
(392, 548)
8800TL83N1WBB116100001-01.jpg
(428, 549)
8800TL83J29BB1B1100002-01.jpg
(502, 544)
8800TL83M3MBB12E100001-01.jpg
(233, 537)
8800TL83K0ZAB127100002-01.jpg
(241, 602)
8800TL83M3MAD111100001-01.jpg
(141, 531)
8800TL83M3LAZ1A4100001-01.jpg
(380, 612)
8800TL83H38AL12E100001-01.jpg
(228, 586)
8800TL83N0SAS184100001-01.jpg
(505, 586)
8800TL83J10AR114100001-01.jpg
(456, 512)
8800TL83H1WAB113100001-01.jpg
(440, 650)
8800TL83H0TAV12J100001-01.jpg
(357, 570)
8800TL8

(126, 225)
8800TL83H1WBD101100001-01.jpg
(411, 650)
8800TL83J29AC11K100001-01.jpg
(288, 24)
8800TL83F2RAT1BE100002-01.jpg
(374, 659)
8800TL83H1GAS108100001-01.jpg
(428, 643)
8800TL83N0VAN10B100001-01.jpg
(273, 534)
8800TL83J1CAB191100001-01.jpg
(576, 679)
8800TL83Q15AF15H100001-01.jpg
(484, 552)
8800TL83H1GAN1AB100001-01.jpg
(456, 686)
8800TL83H1XAN1AB100001-03.jpg
(386, 562)
8800TL83G1LAV102100001-01.jpg
(472, 638)
8800TL83H0MAC16J100001-01.jpg
(266, 619)
8800TL83H0MAD12H100001-01.jpg
(163, 524)
8800TL83G1KAE16H100002-01.jpg
(534, 584)
8800TL83U1UAN166100001-01.jpg
(253, 616)
8800TL83L1CAG12L100001-01.jpg
8800TL83F2RAS19B100002-01.jpg
8800TL83J1CAV114100001-01.jpg
(340, 720)
8800TL83J2CAK104100001-01.jpg
8800TL83J10BD145100001-01.jpg
(386, 421)
8800TL83N0ZAU163100001-01.jpg
(308, 490)
8800TL83U1UAM175100001-01.jpg
(280, 646)
8800TL83N2JAJ14K100001-01.jpg
(293, 584)
8800TL83H1GAP133100001-01.jpg
(410, 745)
8800TL83H1XAG135100001-01.jpg
(404, 825)
8800TL83N0SAJ198100001-01.jpg
(360, 618

(220, 477)
8800TL83H1XAU10D100001-01.jpg
(363, 686)
8800TL83K2MBA196100001-01.jpg
(388, 666)
8800TL83M0JAN10H100001-01.jpg
(208, 602)
8800TL83M0UAE14H100001-01.jpg
(258, 564)
8800TL83H1QAF154100001-01.jpg
(386, 935)
8800TL83R33AD11B100001-01.jpg
(336, 593)
8800TL83E0KAB12H100001-01.jpg
(566, 514)
8800TL83N1TAG107100001-01.jpg
(275, 551)
8800TL83M0BAH1AL100001-01.jpg
(280, 226)
8800TL83N18AY18L100001-01.jpg
(403, 706)
8800TL83N0SAA11F100001-01.jpg
8800TL83N0SBC1B6100001-01.jpg
(372, 744)
8800TL83D38AJ151100001-01.jpg
(574, 575)
8800TL83J2CAJ15C100001-01.jpg
(343, 677)
8800TL83E2QAT113100001-01.jpg
(378, 673)
8800TL83H36AA154100001-01.jpg
8800TL83G1LAV173100002-01.jpg
(414, 610)
8800TL83H1GBA1B2100001-01.jpg
(522, 595)
8800TL83F2RAT1B3100001-01.jpg
(506, 672)
8800TL83F2RAA16D100003-01.jpg
8800TL83N2FAP12N100001-01.jpg
(263, 522)
8800TL83M3LAR112100001-01.jpg
(408, 456)
8800TL83G1LAJ1AK100001-01.jpg
(310, 158)
8800TL83G1KAT10E100001-01.jpg
(333, 691)
8800TL83F2RAB1BJ100001-01.jpg
(392, 21

---------------------------------------------------------------------------------------

In [ ]:
# create result2 dir
import shutil

# img_path = "/home/kelly/Pictures/fastrcnn/8800_0302_100/demo"
# xml_path = "/home/kelly/Pictures/fastrcnn/8800_0302_100/result3/data"
# output_path = "/home/kelly/Pictures/fastrcnn/8800_0302_100/result3"

# for root, dirs, files in os.walk(xml_path):
#     for file in files:
#         shutil.move(os.path.join(root,file),os.path.join(output_path,"Annotations",file))
#         shutil.copy(os.path.join(img_path, root.split("/")[-1], file[:-4]+".jpg"),\
#                     os.path.join(output_path,"JPEGImages",file[:-4]+".jpg"))
#         shutil.copy(os.path.join(img_path, root.split("/")[-1], file[:-4]+".jpg"),\
#                     os.path.join(xml_path, root.split("/")[-1], file[:-4]+".jpg"))
# merge last dir
last_dir = "/home/kelly/Pictures/fastrcnn/8800_0302_100/result2"
now_dir = "/home/kelly/Pictures/fastrcnn/8800_0302_100/result3"
for root,dirs,files in os.walk(os.path.join(last_dir,"Annotations")):
    for file in files:
        print(file)
        shutil.copy(os.path.join(root,file),os.path.join(now_dir,"Annotations",file))
        root_tmp = os.path.join(*(root.split("/")[:-1]))
        shutil.copy(os.path.join('/'+root_tmp,"JPEGImages",file[:-4]+".jpg"),\
                    os.path.join(now_dir,"JPEGImages",file[:-4]+".jpg"))
        


# use the crop image to find no rec and save with xml
#### xml is include x y xmax ymax

In [ ]:
import os,cv2
import numpy as np
import xml.etree.cElementTree as ET
import shutil


crop_image_path = "/home/kelly/Pictures/defect/0308_work/2800_DBC_0928-0305"
no_rec_path = "/home/kelly/Pictures/defect/0308_work/no_redRec"
xml_path = "/home/kelly/Pictures/defect/0308_work/xml"

no_rec_arr = os.listdir(no_rec_path)

#8800TL7BQ2NBC138100001-01

for root,dirs,files in os.walk(crop_image_path):
    for file in files:
        print(file)
        if file in no_rec_arr:
            print("!!")
            # create xml
            oriImg = cv2.imread(os.path.join(no_rec_path,file[:-4]+".jpg"))
            if not os.path.exists(os.path.join("/home/kelly/Pictures/defect/0308_work/data",root.split('/')[-1])):
                os.mkdir(os.path.join("/home/kelly/Pictures/defect/0308_work/data",root.split('/')[-1]))
            shutil.copy(os.path.join(no_rec_path,file[:-4]+".jpg"), os.path.join("/home/kelly/Pictures/defect/0308_work/data",root.split('/')[-1],file[:-4]+".jpg"))
            cropImg = cv2.imread(os.path.join(root,file[:-4]+".jpg"))
            
            result = cv2.matchTemplate(oriImg, cropImg, cv2.TM_CCOEFF_NORMED) 
            x_and_y = np.unravel_index(result.argmax(),result.shape)
            print(x_and_y)
            tree = ET.parse('/home/kelly/Pictures/fastrcnn/8800_0307/result/Annotations/8800AAHFJABXD1A2100001-01.xml')
            tree_root = tree.getroot()
            change_dict = {"filename":"8800TL76B0TAM1BA100001-01","width":oriImg.shape[1],"height":oriImg.shape[0],"xmin":x_and_y[1],"ymin":x_and_y[0],\
                                           "xmax":x_and_y[1]+cropImg.shape[1],"ymax":x_and_y[0]+cropImg.shape[0],"name":root.split('/')[-1]}

            for key in change_dict.keys():
                for e in tree_root.iter(key):
                    e.text = str(change_dict[key])
                    tree.write(os.path.join(xml_path,file[:-4]+".xml"))
        else:
            print("the file doesn't exist")
            

### -----------------------------------------------------------------------------------------

In [9]:
# find red rec and move to the data and delete xml
# PPP0 PC07 PF01 PF02
import os,shutil
base_path = "/msl/home/kelly08385/Pictures/Fast_defect/2800/ori"
des_path = "/msl/home/kelly08385/Pictures/Fast_defect/2800/rec"
# xml_path = "/home/kelly/Pictures/fastrcnn/2300_0322/result/Annotations"

for root,dirs,files in os.walk(base_path):
    for file in files:
        dir = root.split('/')[-1]
        cropImg = getImgInRedRect(os.path.join(root,file))
        if not cropImg is None:
            print(os.path.join(root,file))
            _despath = os.path.join(des_path,dir)
            mkdir(_despath)
#             print(os.path.join(xml_path,file[:-4]+".xml"))
            shutil.move(os.path.join(root,file),os.path.join(_despath,file))
#             os.remove(os.path.join(xml_path,file[:-4]+".xml"))


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:34: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.


/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/PS_P/2800TL7913NAA19J100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/PS_P/2800TL7BD2KAW1PA100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/PS_P/2800TL7BH32BD1D5100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/PS_P/2800TL78V34AJ1J5100001-03.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/PS_P/2800TL78J0YBD1DE100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/PS_P/2800TL78J22AZ196100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/PS_P/2800TL7BL07AK1P2100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/PS_P/2800TL7840UAT182100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/PS_P/2800TL7822DAF14J100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/PS_P/2800TL7CA1SBD18C100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/PS_P/2800TL78P0RBD1T1100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/PS_

/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/PS_P/2800TL7BQ1LBB115100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/PS_P/2800TL7BF2PBD183100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/PS_P/2800TL7830EAC1AF100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/PS_P/2800TL7AH0UAY169100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/PS_P/2800TL7BG0KAN132100002-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/PS_P/2800AAHL5BKXH171100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/PS_P/2800TL78Q15BA1A6100001-03.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/PS_P/2800TL77R20AD15G100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/PS_P/2800TL7830HBB1B1100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/PS_P/2800TL7BL04AS161100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/PS_P/2800TL7AF2GAU11G100003-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/PS_

/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/PS_P/2800TL78V0GAH127100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/PS_P/2800TL7860SAK1C5100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/PS_P/2800TL7BC0TBC1K1100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/PS_P/2800TL78S26BB149100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/PS_P/2800TL7BC2QAV1MA100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/PS_P/2800TL78W1EBB141100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/PS_P/2800TL78U36AN171100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/PS_P/2800TL7AK08BD17D100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/PS_P/2800TL7AF2GAK18G100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/PS_P/2800TL7860SAQ1AE100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/PS_P/2800TL7CA36AR1KA100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/PS_

/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/PS_P/2800TL7BF2RAB1F8100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/PS_P/2800TL78J0ZAQ109100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/PS_P/2800TL78329AZ1BK100002-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/PS_P/2800TL7AD05AB1DF100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/PS_P/2800TL7BC2QAC1A7100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/PS_P/2800TL78T29AC19G100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/PS_P/2800TL78V2DAZ1MC100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/PS_P/2800TL7921WAQ18K100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/PS_P/2800TL7BB0DAY112100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/PS_P/2800TL7BL04AW1C6100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/PS_P/2800TL7BG10BD178100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/PS_

...

/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/C010/2800TL7BG0KAP1F5100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/C010/2800TL7BK2KAX1Q5100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/C010/2800TL7BF2QAQ1P8100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/C010/2800TL77R20AT1A5100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/C010/2800TL7BD3EAK1N9100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/C010/2800TL7BD2FAQ1Q6100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/C010/2800AAJBDAMXN121100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/C010/2800TL7BB3WAW10A100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/C010/2800TL77U20AH17G100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/C010/2800TL7BA23AN125100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/C010/2800TL7BQ18AQ1N1100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/C01

/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/C010/2800TL7BK2LAY192100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/C010/2800TL7B92EAN1F7100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/C010/2800TL7BG0KAD1N9100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/C010/2800TL7BD2FBC105100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/C010/2800TL7BL4GAJ1A3100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/C010/2800TL7BL1QAM1L2100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/C010/2800TL7BH0UAW184100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/C010/2800TL7BB0DAF167100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/C010/2800TL7BC1ZAH178100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/C010/2800TL7BQ15AC1CA100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/C010/2800TL7BL06BB14A100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/C01

/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/C010/2800TL7BG2LAR104100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/C010/2800TL7BJ39AN1D1100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/C010/2800TL7BK2KAQ10A100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/C010/2800TL7BG0KAV183100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/C010/2800TL7B92HAR1GA100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/C010/2800TL7BB3UAM138100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/C010/2800TL77Q19AQ1A1100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/C010/2800TL7BL04AY1Q5100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/C010/2800TL7BM0ZBC133100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/C010/2800TL7BG2ZAF1A2100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/C010/2800TL7CD3YBD12A100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/C01

/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/C010/2800TL7BE2ZBC153100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/C010/2800TL7BC0VBA1H3100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/C010/2800TL7BA2HAN123100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/C010/2800TL77R03AU178100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/C010/2800TL7BE2YAC1F2100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/C010/2800TL7BG11AL1J5100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/C010/2800TL7BG0NAT1AA100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/C010/2800TL7BD2KAL11A100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/C010/2800TL7BH0UAZ18A100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/C010/2800TL7BC20AQ101100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/C010/2800TL7BH3TBC1NA100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/C01

/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/C010/2800TL7BG0MAX1G1100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/C010/2800TL7BG3RBC1P2100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/C010/2800TL7BG2GAW1K2100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/C010/2800TL7B92GBA1N8100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/C010/2800TL7BD0HAK1H4100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/C010/2800TL7BE1WAU167100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/C010/2800TL7BA27BA1F4100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/C010/2800TL7CA36AW191100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/C010/2800TL7BA2WAK1F4100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/C010/2800TL7BD3BAD197100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/C010/2800TL7BA28AH122100001-01.jpg
/msl/home/kelly08385/Pictures/Fast_defect/2800/ori/C01

In [25]:
# all defect class to one class
# change xml object name to defect

source_xml_path = "/msl/home/kelly08385/Pictures/trained_defect/Z695/Annotations"
des_xml_path = "/msl/home/kelly08385/Pictures/trained_defect/total_0416/Annotations"
source_img_path = "/msl/home/kelly08385/Pictures/trained_defect/Z695/data"
des_img_path = "/msl/home/kelly08385/Pictures/trained_defect/total_0416/JPEGImages"
#des_img_path2 = "/msl/home/kelly08385/Pictures/trained_defect/total_0416/data/1156"

for root,dirs,files in os.walk(source_img_path):
    for file in files:
        dir = root.split('/')[-1]
        mkdir(os.path.join(des_img_path2,dir))
        _source_xml_path = os.path.join(source_xml_path,file[:-4]+'.xml')
        if os.path.exists(_source_xml_path):
            tree = ET.parse(_source_xml_path)
            tree_root = tree.getroot()
            change_dict = {"name":'defect'}

            for key in change_dict.keys():
                for e in tree_root.iter(key):
                    e.text = str(change_dict[key])
                    tree.write(os.path.join(des_xml_path,file[:-4]+".xml"))
            shutil.copy(os.path.join(root,file),os.path.join(des_img_path,file))
            #shutil.copy(os.path.join(root,file),os.path.join(des_img_path2,dir,file))